# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [34]:
class RealEstateEnv(Env):
    def __init__(self):
        # Actions we can take, lower price, same price, increase price
        self.action_space = Discrete(3)
        # House prices array
        self.observation_space = Box(low=np.array([100000]), high=np.array([1100000]))
        # Set start price
        self.state = 450000 + random.randint(-100000,100000)
        # Set finding length
        self.finding_length = 180
        
        self.low_price = random.randint(300000, 500000)
        self.high_price = random.randint(self.low_price, (int)(self.low_price * 1.1))
        
    def step(self, action):
        # Apply action
        if action == 0:
            self.state -= 10000
        elif action == 2:
            self.state += 10000
        
        # Reduce house finding length by 1 second
        self.finding_length -= 1
        
        # Calculate reward
        if self.state >= self.low_price and self.state <= self.high_price: 
            reward = 1
        else:
            reward = -1
        
        # Check if finding time is over
        if self.finding_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply price fluctuations
        # self.state += random.randint(-10000,25000)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 450000 + random.randint(-100000,100000)
        # Reset shower time
        self.finding_length = 180
        return self.state

In [35]:
env = RealEstateEnv()

In [36]:
env.observation_space.sample()

array([326847.97], dtype=float32)

In [37]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-136
Episode:2 Score:-54
Episode:3 Score:-180
Episode:4 Score:-170
Episode:5 Score:-140
Episode:6 Score:-168
Episode:7 Score:-180
Episode:8 Score:-180
Episode:9 Score:-130
Episode:10 Score:-160


# 2. Create a Deep Learning Model with Keras

In [38]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [39]:
states = env.observation_space.shape
actions = env.action_space.n

In [40]:
actions

3

In [43]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [42]:
del model 

NameError: name 'model' is not defined

In [44]:
model = build_model(states, actions)

In [45]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [46]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [47]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 37s 4ms/step - reward: -0.8216
55 episodes - episode_reward: -147.673 [-180.000, -30.000] - loss: 42611046.483 - mae: 75476.734 - mean_q: 123606.271

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 38s 4ms/step - reward: -0.8224
56 episodes - episode_reward: -148.179 [-180.000, -36.000] - loss: 169574304.000 - mae: 135953.484 - mean_q: 238893.266

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 39s 4ms/step - reward: -0.8362
55 episodes - episode_reward: -150.218 [-180.000, -2.000] - loss: 522897472.000 - mae: 220410.234 - mean_q: 408845.375

Interval 4 (30000 steps performed)
 6458/10000 [==================>...........] - ETA: 14s - reward: -0.7990

In [22]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -160.000, steps: 180
Episode 2: reward: -166.000, steps: 180
Episode 3: reward: -170.000, steps: 180
Episode 4: reward: -168.000, steps: 180
Episode 5: reward: -174.000, steps: 180
Episode 6: reward: -168.000, steps: 180
Episode 7: reward: -164.000, steps: 180
Episode 8: reward: -174.000, steps: 180
Episode 9: reward: -166.000, steps: 180
Episode 10: reward: -172.000, steps: 180
Episode 11: reward: -170.000, steps: 180
Episode 12: reward: -174.000, steps: 180
Episode 13: reward: -172.000, steps: 180
Episode 14: reward: -170.000, steps: 180
Episode 15: reward: -172.000, steps: 180
Episode 16: reward: -174.000, steps: 180
Episode 17: reward: -170.000, steps: 180
Episode 18: reward: -162.000, steps: 180
Episode 19: reward: -162.000, steps: 180
Episode 20: reward: -174.000, steps: 180
Episode 21: reward: -172.000, steps: 180
Episode 22: reward: -156.000, steps: 180
Episode 23: reward: -170.000, steps: 180
Episode 24: reward: -170.000, steps: 

In [23]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: render() got an unexpected keyword argument 'mode'

# 4. Reloading Agent from Memory

In [24]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [25]:
del model
del dqn
del env

In [26]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

NameError: name 'gym' is not defined

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)